In [1]:
#from __future__ import print_function
import argparse
import os
import imp
import algorithms as alg
from dataloader import DataLoader, GenericDataset

In [2]:
import torch
import torch.nn as nn
torch.cuda.device_count()

1L

In [3]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [4]:
args_opt = Namespace(
    exp = 'Vae_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features',
    evaluate = False,
    checkpoint = 4,
    num_workers = 1,
    cuda = True,
    disp_step= 5
)

In [5]:
exp_config_file = os.path.join('.','config',args_opt.exp+'.py')
# if args_opt.semi == -1:
exp_directory = os.path.join('.','experiments',args_opt.exp)
# else:
#    assert(args_opt.semi>0)
#    exp_directory = os.path.join('.','experiments/unsupervised',args_opt.exp+'_semi'+str(args_opt.semi))

In [6]:
exp_config_file

'./config/Vae_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features.py'

In [7]:
# Load the configuration params of the experiment
print('Launching experiment: %s' % exp_config_file)
config = imp.load_source("",exp_config_file).config
config['exp_dir'] = exp_directory # the place where logs, models, and other stuff will be stored
print("Loading experiment %s from file: %s" % (args_opt.exp, exp_config_file))
print("Generated logs, snapshots, and model files will be stored on %s" % (config['exp_dir']))

Launching experiment: ./config/Vae_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features.py
Loading experiment Vae_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features from file: ./config/Vae_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features.py
Generated logs, snapshots, and model files will be stored on ./experiments/Vae_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features


In [8]:
# Set train and test datasets and the corresponding data loaders
data_train_opt = config['data_train_opt']
data_test_opt = config['data_test_opt']
num_imgs_per_cat = data_train_opt['num_imgs_per_cat'] if ('num_imgs_per_cat' in data_train_opt) else None

In [9]:
dataset_train = GenericDataset(
    dataset_name=data_train_opt['dataset_name'],
    split=data_train_opt['split'],
    random_sized_crop=data_train_opt['random_sized_crop'],
num_imgs_per_cat=num_imgs_per_cat)

custom


In [10]:
dataset_test = GenericDataset(
    dataset_name=data_test_opt['dataset_name'],
    split=data_test_opt['split'],
    random_sized_crop=data_test_opt['random_sized_crop'])

custom


In [11]:
dloader_train = DataLoader(
    dataset=dataset_train,
    batch_size=data_train_opt['batch_size'],
    unsupervised=data_train_opt['unsupervised'],
    epoch_size=data_train_opt['epoch_size'],
    num_workers=args_opt.num_workers,
shuffle=True)

In [12]:
dloader_test = DataLoader(
    dataset=dataset_test,
    batch_size=data_test_opt['batch_size'],
    unsupervised=data_test_opt['unsupervised'],
    epoch_size=data_test_opt['epoch_size'],
    num_workers=args_opt.num_workers,
shuffle=False)

In [13]:
# import torchvision.datasets as datasets
# import torchvision.transforms as transforms

# trainDataDir = '/scratch/um367/DL/data/sampledata/supervised/train'
# valDataDir = '/scratch/um367/DL/data/sampledata/supervised/val'

# kwargs = {'num_workers': 1, 'pin_memory': True} if args_opt.cuda else {}

# dloader_train = torch.utils.data.DataLoader(
#     datasets.ImageFolder(trainDataDir, transform=transforms.ToTensor()),
#     batch_size=data_train_opt['batch_size'], shuffle=True, **kwargs)

# dloader_test = torch.utils.data.DataLoader(
#     datasets.ImageFolder(valDataDir, transform=transforms.ToTensor()),
#     batch_size=data_test_opt['batch_size'], shuffle=True, **kwargs)

In [14]:
len(dloader_train.dataset)

64000

In [15]:
config['disp_step'] = args_opt.disp_step

In [16]:
algorithm = getattr(alg, config['algorithm_type'])(config)

2019-05-15 05:26:52,832 - algorithms.Algorithm - INFO   - Algorithm options {'max_num_epochs': 40, 'disp_step': 5, 'algorithm_type': 'FeatureClassificationModel', 'criterions': {'loss': {'opt': None, 'ctype': 'CrossEntropyLoss'}}, 'data_test_opt': {'unsupervised': False, 'batch_size': 150, 'epoch_size': None, 'random_sized_crop': False, 'split': 'supervised/val', 'dataset_name': 'custom'}, 'exp_dir': './experiments/Vae_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features', 'out_feat_keys': ['conv5'], 'networks': {'classifier': {'opt': [{'cls_type': 'Vae', 'num_classes': 1000, 'nChannels': 256}], 'def_file': 'architectures/MultipleNonLinearClassifiers.py', 'optim_params': {'LUT_lr': [(10, 0.01), (20, 0.002), (30, 0.0004), (40, 8e-05)], 'nesterov': True, 'optim_type': 'sgd', 'lr': 0.1, 'weight_decay': 0.0005, 'momentum': 0.9}, 'pretrained': None}, 'feat_extractor': {'opt': {'num_classes': 4}, 'def_file': 'architectures/vaecnn.py', 'optim_params': None, 'pretrained': './vaetest3.pth.tar'

init_all_networks()
[('classifier', {'opt': [{'cls_type': 'Vae', 'num_classes': 1000, 'nChannels': 256}], 'def_file': 'architectures/MultipleNonLinearClassifiers.py', 'optim_params': {'LUT_lr': [(10, 0.01), (20, 0.002), (30, 0.0004), (40, 8e-05)], 'nesterov': True, 'optim_type': 'sgd', 'lr': 0.1, 'weight_decay': 0.0005, 'momentum': 0.9}, 'pretrained': None}), ('feat_extractor', {'opt': {'num_classes': 4}, 'def_file': 'architectures/vaecnn.py', 'optim_params': None, 'pretrained': './vaetest3.pth.tar'})]


2019-05-15 05:26:53,970 - algorithms.Algorithm - INFO   - Set network feat_extractor
2019-05-15 05:26:53,973 - algorithms.Algorithm - INFO   - ==> Initiliaze network feat_extractor from file architectures/vaecnn.py with opts: {'num_classes': 4}
2019-05-15 05:26:54,424 - algorithms.Algorithm - INFO   - ==> Load pretrained parameters from file ./vaetest3.pth.tar:
2019-05-15 05:27:11,525 - algorithms.Algorithm - INFO   - Initialize criterion[loss]: CrossEntropyLoss with options: None


init_all_criterions()
allocate_tensors()


In [17]:
# device = torch.device("cuda")
# device
256*6*6

9216

In [18]:
if args_opt.cuda: # enable cuda
#     if torch.cuda.device_count() > 1:
#         algorithm = nn.DataParallel(algorithm)
    algorithm.load_to_gpu()
#     algorithm.to(device)
if args_opt.checkpoint > 0: # load checkpoint
    algorithm.load_checkpoint(args_opt.checkpoint, train= (not args_opt.evaluate))

2019-05-15 05:27:11,749 - algorithms.Algorithm - INFO   - Load checkpoint of epoch 4
2019-05-15 05:27:12,472 - algorithms.Algorithm - INFO   - Initialize optimizer: sgd with params: {'LUT_lr': [(10, 0.01), (20, 0.002), (30, 0.0004), (40, 8e-05)], 'nesterov': True, 'optim_type': 'sgd', 'lr': 0.1, 'weight_decay': 0.0005, 'momentum': 0.9} for netwotk: classifier


In [ ]:
if not args_opt.evaluate: # train the algorithm
    algorithm.solve(dloader_train, dloader_test)
else:
    algorithm.evaluate(dloader_test) # evaluate the algorithm


2019-05-15 05:27:13,158 - algorithms.Algorithm - INFO   - Training epoch [  5 /  40]
2019-05-15 05:27:13,161 - algorithms.Algorithm - INFO   - ==> Set to classifier optimizer lr = 0.0100000000
2019-05-15 05:27:13,163 - algorithms.Algorithm - INFO   - Training: Vae_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features
 38%|███▊      | 164/427 [08:32<14:10,  3.24s/it]2019-05-15 05:35:48,040 - algorithms.Algorithm - INFO   - ==> Iteration [  5][ 165 /  426]: {'process_time': 79.794, 'prec5_c1': 12.0202, 'loss': 5.8573, 'prec1_c1': 4.2747, 'load_time': 20.206}


 78%|███████▊  | 334/427 [17:49<04:34,  2.95s/it]2019-05-15 05:45:06,063 - algorithms.Algorithm - INFO   - ==> Iteration [  5][ 335 /  426]: {'process_time': 81.1452, 'prec5_c1': 12.1572, 'loss': 5.8578, 'prec1_c1': 4.1692, 'load_time': 18.8548}


100%|██████████| 427/427 [23:05<00:00,  2.91s/it]
2019-05-15 05:50:19,119 - algorithms.Algorithm - INFO   - ==> Training stats: {'process_time': 81.4623, 'prec5_c1': 12.1109, 'loss': 5.8635, 'prec1_c1': 4.1171, 'load_time': 18.5377}
2019-05-15 05:50:20,153 - algorithms.Algorithm - INFO   - Evaluating: Vae_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features
2019-05-15 05:50:20,158 - algorithms.Algorithm - INFO   - ==> Dataset: custom_supervised/val [64000 images]
100%|██████████| 427/427 [19:36<00:00,  2.24s/it]
2019-05-15 06:09:56,624 - algorithms.Algorithm - INFO   - ==> Results: {'process_time': 78.7489, 'prec5_c1': 11.7518, 'loss': 5.992, 'prec1_c1': 4.146, 'load_time': 21.2511}
2019-05-15 06:09:56,646 - algorithms.Algorithm - INFO   - ==> Evaluation stats: {'process_time': 78.7489, 'prec5_c1': 11.7518, 'loss': 5.992, 'prec1_c1': 4.146, 'load_time': 21.2511}
2019-05-15 06:09:56,648 - algorithms.Algorithm - INFO   - Training epoch [  6 /  40]
2019-05-15 06:09:56,649 - algorithms.Al

 52%|█████▏    | 224/427 [11:53<09:45,  2.88s/it]2019-05-15 06:21:53,318 - algorithms.Algorithm - INFO   - ==> Iteration [  6][ 225 /  426]: {'process_time': 81.1062, 'prec5_c1': 13.1911, 'loss': 5.7702, 'prec1_c1': 4.5807, 'load_time': 18.8938}


 92%|█████████▏| 394/427 [20:23<01:44,  3.17s/it]2019-05-15 06:30:22,753 - algorithms.Algorithm - INFO   - ==> Iteration [  6][ 395 /  426]: {'process_time': 80.4958, 'prec5_c1': 13.0903, 'loss': 5.7834, 'prec1_c1': 4.6042, 'load_time': 19.5042}


...

 67%|██████▋   | 284/427 [16:03<08:03,  3.38s/it]2019-05-15 07:04:13,391 - algorithms.Algorithm - INFO   - ==> Iteration [  7][ 285 /  426]: {'process_time': 81.3819, 'prec5_c1': 14.4982, 'loss': 5.693, 'prec1_c1': 5.1673, 'load_time': 18.6181}


100%|██████████| 427/427 [23:39<00:00,  2.53s/it]
2019-05-15 07:11:46,694 - algorithms.Algorithm - INFO   - ==> Training stats: {'process_time': 81.2882, 'prec5_c1': 14.249, 'loss': 5.71, 'prec1_c1': 5.0734, 'load_time': 18.7118}
2019-05-15 07:11:47,677 - algorithms.Algorithm - INFO   - Evaluating: Vae_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features
2019-05-15 07:11:47,681 - algorithms.Algorithm - INFO   - ==> Dataset: custom_supervised/val [64000 images]
100%|██████████| 427/427 [17:44<00:00,  2.20s/it]
2019-05-15 07:29:32,000 - algorithms.Algorithm - INFO   - ==> Results: {'process_time': 76.8415, 'prec5_c1': 12.6823, 'loss': 5.9138, 'prec1_c1': 4.5909, 'load_time': 23.1585}
2019-05-15 07:29:32,021 - algorithms.Algorithm - INFO   - ==> Evaluation stats: {'process_time': 76.8415, 'prec5_c1': 12.6823, 'loss': 5.9138, 'prec1_c1': 4.5909, 'load_time': 23.1585}
2019-05-15 07:29:32,023 - algorithms.Algorithm - INFO   - Training epoch [  8 /  40]
2019-05-15 07:29:32,024 - algorithms.A

 41%|████      | 174/427 [09:34<14:05,  3.34s/it]2019-05-15 07:39:09,096 - algorithms.Algorithm - INFO   - ==> Iteration [  8][ 175 /  426]: {'process_time': 81.5994, 'prec5_c1': 15.4857, 'loss': 5.6109, 'prec1_c1': 5.7067, 'load_time': 18.4006}


 81%|████████  | 344/427 [18:41<04:19,  3.13s/it]2019-05-15 07:48:16,531 - algorithms.Algorithm - INFO   - ==> Iteration [  8][ 345 /  426]: {'process_time': 81.6977, 'prec5_c1': 15.0995, 'loss': 5.6367, 'prec1_c1': 5.5652, 'load_time': 18.3023}


100%|██████████| 427/427 [23:12<00:00,  3.04s/it]
2019-05-15 07:52:44,626 - algorithms.Algorithm - INFO   - ==> Training stats: {'process_time': 81.6378, 'prec5_c1': 14.9578, 'loss': 5.6478, 'prec1_c1': 5.4231, 'load_time': 18.3622}
2019-05-15 07:52:45,357 - algorithms.Algorithm - INFO   - Evaluating: Vae_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features
2019-05-15 07:52:45,361 - algorithms.Algorithm - INFO   - ==> Dataset: custom_supervised/val [64000 images]
 24%|██▎       | 101/427 [05:25<18:18,  3.37s/it]

/scratch/um367/urwa-env/py2.7.12/lib/python2.7/site-packages/torchvision/transforms/functional.py", line 48, in to_tensor